# Примеры использования библиотек Python в работе аналитика данных

## Пример анализа данных с помощью библиотеки Pandas

Для начала загрузим необходимые данные.

Мы будем использовать датасет [Anime Recommendations Database](https://www.kaggle.com/datasets/CooperUnion/anime-recommendations-database?resource=download&select=anime.csv), который содержит рекомендации 76000 пользователей с сайта [myanimelist.net](https://myanimelist.net/).

Импортируем необходимые библиотеки и загрузим файлы в ноутбук.

In [ ]:
import pandas as pd
import numpy as np
anime = pd.read_csv('anime.csv', sep=',')
rating = pd.read_csv('rating.csv', sep=',')

Сначала поработаем с объектом `anime`.

Посмотрим, что у нас за данные. Выведем первые 5 строк:

In [ ]:
anime.head()

Получим размерность:

In [ ]:
anime.shape

Посмотрим информацию об объекте с помощью метода `info()`:

In [ ]:
anime.info()

Теперь проделаем аналогичные действия для другого датафрейма:

In [ ]:
rating.head()

In [ ]:
rating.shape

In [ ]:
rating.info()

Перейдем к описательным статистикам по численным данным наших датафреймов. Выведем общую информацию с помощью метода `describe()`:

In [ ]:
anime.describe()

In [ ]:
rating.describe()

Далее проверим качество наших данных (наличие пропусков, аномалии и т. п.):

In [ ]:
anime.isnull().mean().sort_values(ascending=False)

Видно, что в этом датафрейме `anime` есть пропуски в строках с рейтингом, жанром и типом.

Рассмотрим числовые признаки для данных: минимальное, максимальное, среднее и медианное значения:

In [ ]:
anime.min()

In [ ]:
anime.max()

In [ ]:
anime.mean()

In [ ]:
anime.median()

При верхнеуровневом анализе аномалий в данных мы не заметили. Но на практике лучше более детально проверять данные на выбросы. Например, с помощью гистограммы частот:

In [ ]:
hist = anime['rating'].hist()
figure = hist.get_figure()

Теперь посмотрим на датафрейм `rating`:

In [ ]:
rating.isnull().mean().sort_values(ascending=False)

В этом датафрейме пропусков нет.

In [ ]:
rating.min()

In [ ]:
rating.max()

In [ ]:
rating.mean()

In [ ]:
rating.median()

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

ax.hist(rating['rating'], linewidth=0.5, edgecolor="white")

plt.show()

Полученное распределение *не имеет* нормальный вид, это повод задуматься, почему.

Вернемся к работе с датафреймом `anime`.

Теперь давайте отбросим данные без рейтинга, так как это наша основная метрика для исследования:

In [ ]:
anime = anime.dropna(subset=['rating'])

In [ ]:
anime.isnull().mean().sort_values(ascending=False)

In [ ]:
anime.describe()

Вычистим дубликаты в обоих датафреймах (если они есть):

In [ ]:
anime.drop_duplicates(inplace=True)

In [ ]:
rating.drop_duplicates(inplace=True)

Пусть нам нужно определить уникальное количество аниме по типам (ТВ, фильм и т. п.):

In [ ]:
anime['type'].value_counts()

In [ ]:
anime['type'].value_counts(normalize=True)

In [ ]:
anime.groupby('type')['genre'].value_counts()

А теперь посмотрим среднюю оценку в разбивке по типу и жанрам:

In [ ]:
anime.groupby(['type','genre'])['rating'].mean()

Проведенный анализ данных является заготовкой для передачи результатов заказчику.

## Пример анализа данных из нескольких источников

Мы продолжим работу с датасетом [Anime Recommendations Database](https://www.kaggle.com/datasets/CooperUnion/anime-recommendations-database?resource=download&select=anime.csv).

У нас есть 2 датафрейма: `anime` и `rating`. Давайте теперь их объединим:

In [ ]:
data = rating.merge(anime, left_on='anime_id', right_on='anime_id', suffixes=('_rating', '_anime'))
data

Посмотрим некоторые статистические выкладки:

In [ ]:
data['name'].value_counts(ascending=False)

In [ ]:
data.groupby(['name'])['rating_rating'].mean()

Сравним для примера рейтинг Наруто:

In [ ]:
data[data.name == 'Naruto'].groupby(['name'])['rating_rating'].mean()

Если прочитать описание полей, то станет понятно, что в случае просмотра без постановки оценки в таблицу `rating` проставлялась оценка -1. Это занизило результат.

А теперь давайте соберем агрегацию по названию аниме с количеством пользователей, которые проставили оценку, и медианным рейтингом:

In [ ]:
df = data.groupby(['name']).agg({'rating_rating': ['median'], 'user_id': ['count']}).reset_index()
df

Этот результат можно выгрузить в .csv, настроить внешний вид и представить как отчет по оценкам различных аниме.

## Работа с матрицей корреляции

Для начала вспомним, какой у нас есть набор данных:

In [ ]:
anime = pd.read_csv('anime.csv', sep=',')
anime.head()

Введем важное определение.

**Коэффициент корреляции** — это статистическая мера, которая вычисляет силу связи между относительными движениями двух переменных. Значения коэффициента корреляции находятся в диапазоне от -1.0 до 1.0.

**Основные выводы** в зависимости от коэффициента корреляции:
1. Значения всегда находятся в диапазоне от -1 (сильная отрицательная связь) до +1 (сильная положительная связь).
2. Значения при нулевом значении или близкие к нулю подразумевают слабую или отсутствующую связь.
3. Значения коэффициентов корреляции менее +0,8 или более -0,8 не считаются значимыми.

Теперь посмотроим непосредственно матрицу корреляции:

In [ ]:
anime.corr()

Также можно округлить коэффициенты корреляции:

In [ ]:
anime.corr().round(2)

Визуализируем полученный результат:

In [ ]:
corr = anime.corr()
corr.style.background_gradient(cmap='coolwarm')

Еще один вариант визуализации:

In [ ]:
corr = anime.corr()
corr.style.background_gradient(cmap='RdYlGn')

**Наши выводы.** Значимой корреляции в наших даных не наблюдается.

## Создание интерактивных графиков

Включаем интерактивные графики:

In [1]:
%matplotlib notebook

Посмотрим анимацию простого линейного графика:

In [2]:
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib import rc
import numpy as np
from IPython.display import HTML

x = []
y = []

figure, ax = plt.subplots()

# Задаем границы для осей x и y
ax.set_xlim(0, 100)
ax.set_ylim(0, 10)

line,  = ax.plot(0, 0)

def animation_function(i):
    x.append(i * 10)
    y.append(i)

    line.set_xdata(x)
    line.set_ydata(y)
    return line,

animation = FuncAnimation(figure,
                          func = animation_function,
                          frames = np.arange(0, 10, 0.1),
                          interval = 10)
rc('animation', html='jshtml')
animation

<IPython.core.display.Javascript object>

Теперь посмотрим анимацию для bar chart:

In [3]:
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np

fig = plt.figure(figsize = (7,5))
axes = fig.add_subplot(1,1,1)
axes.set_ylim(0, 300)
palette = ['red', 'orange', 'yellow',
           'green', 'blue', 'purple']

y1, y2, y3, y4, y5, y6 = [], [], [], [], [], []

def animation_function(i):
    y1 = i
    y2 = 3 * i
    y3 = 2 * i
    y4 = 6 * i
    y5 = 5 * i
    y6 = 4 * i

    plt.xlabel("User ID")
    plt.ylabel("Money spent, rub")

    plt.bar(["user_1", "user_2", "user_3",
             "user_4", "user_5", "user_6"],
            [y1, y2, y3, y4, y5, y6],
            color = palette)

plt.title("Bar Chart Animation")


animation = FuncAnimation(fig, animation_function,
                          interval = 50, save_count=100)

rc('animation', html='jshtml')
animation

Output hidden; open in https://colab.research.google.com to view.

А теперь посмотрим визуализацию с помощью библиотеки `plotly`.
Больше примеров работы с библиотекой вы найдете в [документации](https://plotly.com/python/).

Подключаем необходимые библиотеки:

In [ ]:
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

import numpy as np
import pandas as pd

Для начала построим простой график:

In [ ]:
x = np.arange(0, 10, 1.0)
def f(x):
    return x**3

px.line(x=x, y=f(x)).show()

Построим график сразу двух функций:

In [ ]:
fig = go.Figure()
fig.add_trace(go.Line(x=x, y=f(x), name='f1(x)=x^3'))
fig.add_trace(go.Line(x=x, y=x*x, name='f2(x)=x^2'))
fig.show()

Усложним нашу конструкцию:

In [ ]:
fig = go.Figure()
fig.add_trace(go.Line(x=x, y=f(x), mode='lines+markers',  name='f1(x)=x<sup>3</sup>'))
fig.add_trace(go.Line(x=x, y=x*x, mode='markers', name='f2(x)=x<sup>2</sup>'))
fig.update_layout(legend_orientation="v",
                  legend=dict(x=0.5, xanchor="auto"),
                  hovermode = 'x')
fig.update_traces(hoverinfo="all", hovertemplate="Значение аргумента: %{x}<br>Значение функции: %{y}")
fig.show()

А теперь добавим слайдер:

In [ ]:
ticks_number = len(x)
trace_list = [go.Line(visible=True, x=[x[0]], y=[f(x)[0]], mode='lines+markers', name='f(x)=x<sup>3</sup>')]

for i in range(1, len(x)):
    trace_list.append(go.Line(visible=False, x=x[:i+1], y=f(x[:i+1]), mode='lines+markers', name='f(x)=x<sup>3</sup>'))

fig = go.Figure(data=trace_list)

steps = []
for i in range(ticks_number):
    step = dict(
        method = 'restyle',
        args = ['visible', [False] * len(fig.data)],
    )
    step['args'][1][i] = True

    steps.append(step)

sliders = [dict(
    steps = steps,
)]

fig.layout.sliders = sliders

fig.show()

Посмотрим интерактивные bar-чарты:

In [ ]:
a = np.arange(0, 100, 5.0)
def g(a):
    return a**2

px.bar(x=a, y=g(a)).show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=a, y=g(a),  name='g(a)=x<sup>2</sup>'))
fig.update_layout(legend_orientation="h",
                  legend=dict(x=0.5, xanchor="center"),
                  hovermode = 'x')
fig.update_traces(hoverinfo="all", hovertemplate="Значение аргумента: %{x}<br>Значение функции: %{y}")
fig.show()

Рассмотрим пример с выпадающим списком:

In [ ]:
import plotly.graph_objects as px
import numpy as np

Построим для начала простой график по точкам:

In [ ]:
np.random.seed(0)

random_x = np.random.randint(1, 100, 100)
random_y = np.random.randint(1, 100, 100)

fig = go.Figure(data=[px.Scatter(
        x=random_x,
        y=random_y,
        mode='markers',)
])

fig.update_layout(title = "Scatter Plot")

fig.show()

Добавим выпадающий список с возможносью выбрать тип графика:

In [ ]:
np.random.seed(0)

random_x = np.random.randint(1, 100, 100)
random_y = np.random.randint(1, 100, 100)

fig = px.Figure(data=[px.Scatter(
    x=random_x,
    y=random_y,
    mode='markers',)
])

fig.update_layout(
    updatemenus=[
        dict(
        buttons=list([
            dict(
            args=["type", "scatter"],
            label="Scatter Plot",
            method="restyle"
            ),
                        dict(
                        args=["type", "box"],
                        label="Box Plot",
                        method="restyle"
                        ),
            dict(
            args=["type", "bar"],
            label="Bar Plot",
            method="restyle"
            )
            ]),
            direction="down",
            ),
          ],
      title = "Dropdown Menu in plotly Graph Using Python"
)

fig.show()
